In [4]:
# ============================================================
# Kaggle-Optimized Apple DepthPro Evaluation on KITTI
# ============================================================

import torch
import cv2
import numpy as np
from PIL import Image
import matplotlib
matplotlib.use("Agg")   # Important for Kaggle speed
import matplotlib.pyplot as plt
from pathlib import Path
import time
import json
from tqdm import tqdm
from transformers import DepthProImageProcessorFast, DepthProForDepthEstimation

class DepthProKITTITester:
    def __init__(self, device='cuda', use_scale_alignment=False):
        """
        Kaggle-optimized metric depth evaluation for DepthPro.
        """
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.model = None
        self.image_processor = None
        self.use_scale_alignment = use_scale_alignment

        print(f"🚀 Using device: {self.device}")
        print(f"📏 Scale alignment: {'Enabled' if use_scale_alignment else 'Disabled (metric model)'}")

    def load_model(self):
        """Load DepthPro with autocast + GPU"""
        print("📥 Loading Apple DepthPro...")

        try:
            self.image_processor = DepthProImageProcessorFast.from_pretrained("apple/DepthPro-hf")
            self.model = DepthProForDepthEstimation.from_pretrained("apple/DepthPro-hf").to(self.device)
            self.model.eval()
            print("✅ Model loaded successfully!")
            return True

        except Exception as e:
            print(f"❌ Error loading model: {e}")
            return False

    def load_kitti_depth(self, depth_path):
        """Load KITTI ground truth depth (fast)"""
        depth = cv2.imread(str(depth_path), cv2.IMREAD_ANYDEPTH).astype(np.float32)
        depth /= 256.0
        valid_mask = depth > 0
        return depth, valid_mask

    def predict_depth(self, image_path):
        """Fast inference using autocast on GPU"""
        start_time = time.time()

        image = Image.open(str(image_path)).convert("RGB")
        original_size = (image.height, image.width)

        # GPU preprocessing
        inputs = self.image_processor(images=image, return_tensors="pt").to(self.device)

        # 🔥 Mixed precision for 2–3× speedup
        with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
            outputs = self.model(**inputs)

        post = self.image_processor.post_process_depth_estimation(
            outputs, target_sizes=[original_size]
        )

        depth_map = post[0]["predicted_depth"]

        if isinstance(depth_map, torch.Tensor):
            depth_map = depth_map.detach().cpu().numpy()

        inference_time = time.time() - start_time
        return depth_map, inference_time, {}

    def align_prediction_to_gt(self, pred, gt, valid_mask):
        pred_valid = pred[valid_mask]
        gt_valid = gt[valid_mask]

        A = np.vstack([pred_valid, np.ones(len(pred_valid))]).T
        s, t = np.linalg.lstsq(A, gt_valid, rcond=None)[0]

        return s * pred + t, s, t

    def compute_metrics(self, pred, gt, valid_mask):
        pred = pred[valid_mask]
        gt = gt[valid_mask]

        pred = np.clip(pred, 1e-3, None)
        gt = np.clip(gt, 1e-3, None)

        thresh = np.maximum(gt / pred, pred / gt)
        delta_1 = (thresh < 1.25).mean()
        delta_2 = (thresh < 1.25**2).mean()
        delta_3 = (thresh < 1.25**3).mean()

        rmse = np.sqrt(((pred - gt) ** 2).mean())
        rmse_log = np.sqrt(((np.log(pred) - np.log(gt)) ** 2).mean())
        mae = np.abs(pred - gt).mean()
        abs_rel = (np.abs(pred - gt) / gt).mean()
        sq_rel = (((pred - gt) ** 2) / gt).mean()

        return {
            'delta_1': float(delta_1),
            'delta_2': float(delta_2),
            'delta_3': float(delta_3),
            'rmse': float(rmse),
            'rmse_log': float(rmse_log),
            'mae': float(mae),
            'abs_rel': float(abs_rel),
            'sq_rel': float(sq_rel)
        }

    def visualize_result(self, image_path, pred_depth, gt_depth, valid_mask, metrics, output_path, scale_info):
            """2x2 visualization matching DAV3 style"""
            image = np.array(Image.open(image_path).convert('RGB'))
    
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
            # 1. Input image
            axes[0, 0].imshow(image)
            axes[0, 0].set_title('Input Image')
            axes[0, 0].axis('off')
    
            # 2. Predicted Depth (Metric)
            im1 = axes[0, 1].imshow(pred_depth, cmap='plasma', vmin=0, vmax=80)
            axes[0, 1].set_title('Predicted Depth (Metric)')
            axes[0, 1].axis('off')
            plt.colorbar(im1, ax=axes[0, 1], fraction=0.046, label='meters')
    
            # 3. Ground Truth depth
            gt_disp = gt_depth.copy()
            gt_disp[~valid_mask] = np.nan
            im2 = axes[1, 0].imshow(gt_disp, cmap='plasma', vmin=0, vmax=80)
            axes[1, 0].set_title('Ground Truth Depth')
            axes[1, 0].axis('off')
            plt.colorbar(im2, ax=axes[1, 0], fraction=0.046, label='meters')
    
            # 4. Absolute Error
            err = np.abs(pred_depth - gt_depth)
            err[~valid_mask] = np.nan
            im3 = axes[1, 1].imshow(err, cmap='hot', vmin=0, vmax=10)
            axes[1, 1].set_title('Absolute Error')
            axes[1, 1].axis('off')
            plt.colorbar(im3, ax=axes[1, 1], fraction=0.046, label='meters')
    
            # Metrics box
            text = '\n'.join([f'{k}: {v:.4f}' for k, v in metrics.items() if k != 'inference_time'])
            fig.text(0.02, 0.02, text, fontsize=10, family='monospace',
                     bbox=dict(boxstyle='round', facecolor='wheat'))
    
            plt.tight_layout()
            plt.savefig(output_path, dpi=150, bbox_inches='tight')
            plt.close()
    
            print(f"   💾 Saved: {output_path}")


    def test_on_kitti(self, image_dir, gt_dir, output_dir, num_samples=20):
        image_path = Path(image_dir)
        gt_path = Path(gt_dir)
        output = Path(output_dir)
        output.mkdir(parents=True, exist_ok=True)

        image_files = sorted(image_path.glob("*.png"))
        if not image_files:
            print("❌ No PNG images found!")
            return

        print(f"📁 Found {len(image_files)} images.")
        image_files = image_files[:num_samples]

        all_metrics = []

        for idx, img_path in tqdm(enumerate(image_files), total=len(image_files)):
            pred, t, _ = self.predict_depth(img_path)

            gt_file = gt_path / img_path.name
            gt_depth, valid_mask = self.load_kitti_depth(gt_file)

            pred = cv2.resize(pred, (gt_depth.shape[1], gt_depth.shape[0]))

            if self.use_scale_alignment:
                pred, s, shift = self.align_prediction_to_gt(pred, gt_depth, valid_mask)

            metrics = self.compute_metrics(pred, gt_depth, valid_mask)
            metrics['inference_time'] = t
            all_metrics.append(metrics)
            vis_path = output / f"result_{idx:03d}_{img_path.stem}.png"
            self.visualize_result(
                img_path,
                pred,
                gt_depth,
                valid_mask,
                metrics,
                vis_path,
                scale_info={'scale':1.0, 'shift':0.0}
            )


        # Save metrics
        results_path = output / "metrics.json"
        json.dump({
            "average": {k: float(np.mean([m[k] for m in all_metrics])) for k in all_metrics[0]},
            "per_image": all_metrics
        }, open(results_path, "w"), indent=2)

        print(f"💾 Saved metrics to: {results_path}")

        


# =====================================================
# MAIN (Kaggle paths)
# =====================================================

IMAGE_DIR = "/kaggle/input/images-test"
GT_DIR = "/kaggle/input/ground-truth"
OUTPUT_DIR = "/kaggle/working/depthpro_results"
NUM_SAMPLES = 20

tester = DepthProKITTITester(device="cuda", use_scale_alignment=False)

if tester.load_model():
    tester.test_on_kitti(IMAGE_DIR, GT_DIR, OUTPUT_DIR, NUM_SAMPLES)

print("\n✅ Done!")


🚀 Using device: cuda
📏 Scale alignment: Disabled (metric model)
📥 Loading Apple DepthPro...
✅ Model loaded successfully!
📁 Found 20 images.


  0%|          | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_48/26217687.py:64: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
  5%|▌         | 1/20 [00:03<00:59,  3.15s/it]

   💾 Saved: /kaggle/working/depthpro_results/result_000_1.png


 10%|█         | 2/20 [00:06<00:54,  3.04s/it]

   💾 Saved: /kaggle/working/depthpro_results/result_001_10.png


...

   💾 Saved: /kaggle/working/depthpro_results/result_017_7.png


 95%|█████████▌| 19/20 [00:58<00:03,  3.13s/it]

   💾 Saved: /kaggle/working/depthpro_results/result_018_8.png


100%|██████████| 20/20 [01:02<00:00,  3.10s/it]

   💾 Saved: /kaggle/working/depthpro_results/result_019_9.png
💾 Saved metrics to: /kaggle/working/depthpro_results/metrics.json

✅ Done!


In [5]:
import shutil
from pathlib import Path

OUTPUT_DIR = "/kaggle/working/depthpro_results"
ZIP_PATH = "/kaggle/working/depthpro_results"

print("📦 Creating ZIP file...")

# Create ZIP of entire directory
shutil.make_archive(ZIP_PATH, 'zip', OUTPUT_DIR)

print(f"✅ ZIP created at: {ZIP_PATH}.zip")
print("You can now download the file from the Kaggle sidebar (Output > Files).")


📦 Creating ZIP file...
✅ ZIP created at: /kaggle/working/depthpro_results.zip
You can now download the file from the Kaggle sidebar (Output > Files).
